In [1]:
import numpy as np
import gurobipy as gbp
import time
np.random.seed(352)


In [2]:
n_com=2
n_sup=2
n_dem=3

In [3]:
Si=np.array([[100,200],[200,700]])
Dj=np.array([[75,190],[150,320],[200,225]])
Si,Dj

(array([[100, 200],
        [200, 700]]), array([[ 75, 190],
        [150, 320],
        [200, 225]]))

In [15]:
V_cap=150
V_num=np.array([5,7])
V_cost=np.array([1000,1000])

In [59]:
Dj

array([[ 75, 190],
       [150, 320],
       [200, 225]])

In [5]:
# Cij = Cij.reshape(matrix_rows,matrix_cols)
# Si = Si.reshape(matrix_rows,1)
# Dj = Dj.reshape(matrix_cols,1)

# Indices & Variable Names
supply_nodes = n_sup
demand_nodes = n_dem
supply_nodes_range = range(n_sup)
demand_nodes_range = range(n_dem)
comm_range=range(n_com)
all_nodes_len = n_sup*n_dem
ALL_nodes_range = range(all_nodes_len)

print (supply_nodes_range, demand_nodes_range,comm_range, all_nodes_len)

range(0, 2) range(0, 3) range(0, 2) 6


In [6]:
Pc=Dj/Dj.sum(axis=0)
Pc

array([[0.17647059, 0.2585034 ],
       [0.35294118, 0.43537415],
       [0.47058824, 0.30612245]])

In [17]:
# Create Model, Set MIP Focus, Add Variables, & Update Model
m = gbp.Model(' -- The Transportation Problem -- ')
# Set MIP Focus to 2 for optimality
m.setParam('MIPFocus', 1)
m.setParam(gbp.GRB.Param.PoolSearchMode, 1)
m.setParam(gbp.GRB.Param.PoolGap, 0.10)
decision_var = []
vehicles_var=[]
for orig in supply_nodes_range:
    decision_var.append([])
    vehicles_var.append([])
    for dest in demand_nodes_range:
        decision_var[orig].append([])
        vehicles_var[orig].append(m.addVar(vtype=gbp.GRB.INTEGER,
                                          name='S'+str(orig+1)+'_D'+str(dest+1)+'_V'))
        for comm in comm_range:
#             print (comm,decision_var)
            decision_var[orig][dest].append(m.addVar(vtype=gbp.GRB.INTEGER, 
#                                         obj=Cij[orig][dest],
#                                            obj=1,
                                        name='S'+str(orig+1)+'_D'+str(dest+1)+'_c'+str(comm+1)))
# Update Model Variables
m.update()       


Changed value of parameter MIPFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter PoolSearchMode to 1
   Prev: 0  Min: 0  Max: 2  Default: 0
Changed value of parameter PoolGap to 0.1
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [21]:
m.setObjective(
    gbp.quicksum(gbp.quicksum((int(Dj[dest][comm])-gbp.quicksum(decision_var[orig][dest][comm] for orig in supply_nodes_range))*(Pc[dest][comm])
                            for dest in demand_nodes_range) for comm in comm_range)+
    gbp.quicksum(vehicles_var[orig][dest] for dest in demand_nodes_range),
                        gbp.GRB.MINIMIZE)

In [22]:
m.update()

In [26]:
m.display()

Minimize
   <gurobi.LinExpr: 417.60704281712685 + -0.17647058823529413 S1_D1_c1 + -0.2585034013605442 S1_D1_c2 + -0.35294117647058826 S1_D2_c1 + -0.43537414965986393 S1_D2_c2 + -0.47058823529411764 S1_D3_c1 + -0.30612244897959184 S1_D3_c2 + S2_D1_V + -0.17647058823529413 S2_D1_c1 + -0.2585034013605442 S2_D1_c2 + S2_D2_V + -0.35294117647058826 S2_D2_c1 + -0.43537414965986393 S2_D2_c2 + S2_D3_V + -0.47058823529411764 S2_D3_c1 + -0.30612244897959184 S2_D3_c2>
Subject To
   R0 : <gurobi.LinExpr: S1_D1_c1 + S1_D2_c1 + S1_D3_c1> <= 100.0
   R1 : <gurobi.LinExpr: S1_D1_c2 + S1_D2_c2 + S1_D3_c2> <= 200.0
   R2 : <gurobi.LinExpr: S2_D1_c1 + S2_D2_c1 + S2_D3_c1> <= 200.0
   R3 : <gurobi.LinExpr: S2_D1_c2 + S2_D2_c2 + S2_D3_c2> <= 700.0
   R4 : <gurobi.LinExpr: S1_D1_c1 + S2_D1_c1> <= 75.0
   R5 : <gurobi.LinExpr: S1_D1_c2 + S2_D1_c2> <= 190.0
   R6 : <gurobi.LinExpr: S1_D2_c1 + S2_D2_c1> <= 150.0
   R7 : <gurobi.LinExpr: S1_D2_c2 + S2_D2_c2> <= 320.0
   R8 : <gurobi.LinExpr: S1_D3_c1 + S2_D3_c1>

In [40]:
import math

In [44]:
# Add Supply Constraints
for orig in supply_nodes_range:
    for comm in comm_range:
        m.addConstr(gbp.quicksum(decision_var[orig][dest][comm]
                                 for dest in demand_nodes_range) - Si[orig][comm] <= 0)
# Add Demand Constraints
for dest in demand_nodes_range:  
    for comm in comm_range:
        m.addConstr(gbp.quicksum(decision_var[orig][dest][comm] 
                                 for orig in supply_nodes_range) - Dj[dest][comm] <= 0)
#Add vehicle constraints
for orig in supply_nodes_range:
    m.addConstr(gbp.quicksum(decision_var[orig][dest][comm]
                             for dest in demand_nodes_range for comm in comm_range) <= V_cap*V_num[orig])
for orig in supply_nodes_range:
    m.addConstr(gbp.quicksum(vehicles_var[orig][dest] for dest in demand_nodes_range) <= V_num[orig])
    
for orig in supply_nodes_range:
    for dest in demand_nodes_range:
        m.addConstr(sum(decision_var[orig][dest][comm]
                            for comm in comm_range) == V_cap*vehicles_var[orig][dest])

#  Optimize and Print( Results)
m.optimize()
m.write('path.lp')

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 156 rows, 18 columns and 468 nonzeros
Model fingerprint: 0x3b16fb5a
Variable types: 0 continuous, 18 integer (0 binary)
Coefficient statistics:
  Matrix range     [7e-03, 2e+02]
  Objective range  [2e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+03]

Loaded MIP start from previous solve with objective 77.8335

Presolve removed 140 rows and 0 columns
Presolve time: 0.00s
Presolved: 16 rows, 18 columns, 45 nonzeros
Variable types: 0 continuous, 18 integer (2 binary)

Root relaxation: objective 3.578235e+01, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   35.78235    0    4   77.83353   35.78235  54.0%     -    0s
     0     0   45.85582    0    3   77.83353   45.85582  41.1%     -    0s
     0     0   45.85582    0    3   77.83353   45.85

In [45]:
nSolutions = m.SolCount
print (nSolutions)
for e in range(nSolutions):
        m.setParam(gbp.GRB.Param.SolutionNumber, e)
        print('%g ' % m.PoolObjVal, end='')
        print ([(v.Varname,v.x) for v in m.getVars()])
        break

10
77.8335 [('S1_D1_V', 0.0), ('S1_D1_c1', 0.0), ('S1_D1_c2', 0.0), ('S1_D2_V', 1.0), ('S1_D2_c1', 50.0), ('S1_D2_c2', 100.0), ('S1_D3_V', 1.0), ('S1_D3_c1', 50.0), ('S1_D3_c2', 100.0), ('S2_D1_V', 1.0), ('S2_D1_c1', 0.0), ('S2_D1_c2', 150.0), ('S2_D2_V', 2.0), ('S2_D2_c1', 80.0), ('S2_D2_c2', 220.0), ('S2_D3_V', 1.0), ('S2_D3_c1', 120.0), ('S2_D3_c2', 30.0)]


In [ ]:
if (nSolutions >= 4):
    model.setParam(GRB.Param.SolutionNumber, 3)

    print('Selected elements in fourth best solution:')
    print('\t', end='')
    for e in range(2):
        if Elem[e].Xn > .9:
            print(' El%d' % e, end='')
    print('')

In [159]:
selected = {}
Closed = []
for v in m.getVars():
    var = '%s' % v.VarName
    print (v)
#     print ('|  Supply Facility #', var[1:6], 'is shipping', units, \
#                                             'units to Demand Facility #', var[-5:])

<gurobi.Var S1_D1 (value 75.0)>
<gurobi.Var S1_D2 (value 25.0)>
<gurobi.Var S1_D3 (value -0.0)>
<gurobi.Var S2_D1 (value -0.0)>
<gurobi.Var S2_D2 (value 125.0)>
<gurobi.Var S2_D3 (value 75.0)>


In [52]:
selected = {}
Closed = []
for v in m.getVars():
    var = '%s' % v.VarName
    units=int(v.x)
    selected[var] = units
    if (v.Varname[-1]=='V'):
        print ('-'*100)
        print( '|  Supply Facility #', var[:2], 'is sending', units, \
              'vehicles to Demand Facility #', var[3:5]) 
        
    if v.Varname[-2]=='c':
        print( '|  Supply Facility #', var[:2], 'is shipping', units, \
                                            'units of commodity',var[-2:], 'to Demand Facility #', var[3:5])
    else:
        Closed.append([var[1:6], var[-5:]])
print( '******************************************************************************')
print( '    | Objective Value --------------------- ', int(m.objVal))
print( '    | Supply Facilities ------------------- ', len(Si))
print( '    | Total Supply Units ------------------ ', Si.sum())
print( '    | Demand Facilites -------------------- ', len(Dj))
print( '    | Total Demand Units ------------------ ', Dj.sum())
print( '    | Total Potential Combinations -------- ', len(Si)*len(Dj))
print( '    | Actual Combinations  ---------------- ', len(selected))
# print( '    | Real Time to Optimize (sec.) -------- ', t2)
print( '******************************************************************************')
print( '  --  The Transportation Simplex with Gurobi --')


----------------------------------------------------------------------------------------------------
|  Supply Facility # S1 is sending 0 vehicles to Demand Facility # D1
|  Supply Facility # S1 is shipping 0 units of commodity c1 to Demand Facility # D1
|  Supply Facility # S1 is shipping 0 units of commodity c2 to Demand Facility # D1
----------------------------------------------------------------------------------------------------
|  Supply Facility # S1 is sending 1 vehicles to Demand Facility # D2
|  Supply Facility # S1 is shipping 50 units of commodity c1 to Demand Facility # D2
|  Supply Facility # S1 is shipping 100 units of commodity c2 to Demand Facility # D2
----------------------------------------------------------------------------------------------------
|  Supply Facility # S1 is sending 1 vehicles to Demand Facility # D3
|  Supply Facility # S1 is shipping 50 units of commodity c1 to Demand Facility # D3
|  Supply Facility # S1 is shipping 100 units of commodity c2 t